In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = "/content/sales_data.csv"

In [ ]:
df = pd.read_csv(data)
df = df.sort_index()
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
0,01-01-2022,S001,P0001,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,Winter,0,115
1,01-01-2022,S001,P0002,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,Winter,0,229
2,01-01-2022,S001,P0003,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,Winter,0,157
3,01-01-2022,S001,P0004,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,Winter,0,52
4,01-01-2022,S001,P0005,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,Winter,0,59


In [ ]:
df.columns

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Price', 'Discount',
       'Weather Condition', 'Promotion', 'Competitor Pricing', 'Seasonality',
       'Epidemic', 'Demand'],
      dtype='object')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

In [ ]:
print("Missing values before:", df['Price'].isna().sum())

Missing values before: 0


In [ ]:
# Check stationarity using ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['Demand'])
print("ADF Statistic:", result[0])
print("p-value:", result[1])

if result[1] <= 0.05:
    print("Series is stationary (good for ARIMA).")
else:
    print("Series is NOT stationary. Differencing may be required (d=1).")


ADF Statistic: -12.565004080932258
p-value: 2.0627299881731915e-23
Series is stationary (good for ARIMA).


In [10]:
df

,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
Date,,,,,,,,,,,,,,,
2022-01-01,S001,P0001,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,Winter,0,115
2022-01-01,S004,P0013,Groceries,West,136,104,385,20.24,10,Snowy,0,18.90,Winter,0,110
2022-01-01,S004,P0012,Electronics,West,111,111,113,118.15,0,Snowy,0,133.46,Winter,0,103
2022-01-01,S004,P0011,Clothing,West,195,60,293,52.89,0,Snowy,0,62.29,Winter,0,61
2022-01-01,S004,P0010,Groceries,West,223,120,597,30.02,0,Snowy,0,29.15,Winter,0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-30,S002,P0008,Groceries,South,85,81,94,26.98,0,Snowy,0,24.65,Winter,0,96
2024-01-30,S002,P0007,Toys,South,414,126,0,26.84,10,Snowy,0,29.55,Winter,0,109
2024-01-30,S002,P0006,Clothing,South,98,96,117,63.58,10,Snowy,0,76.22,Winter,0,118


In [11]:
from sklearn.preprocessing import LabelEncoder
cat_cols = [
    'Category',
    'Region',
    'Weather Condition',
    'Seasonality',
]

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))
df[cat_cols].head()

,Category,Region,Weather Condition,Seasonality
Date,,,,
2022-01-01,1,1,2,3
2022-01-01,3,3,2,3
2022-01-01,1,3,2,3
2022-01-01,0,3,2,3
2022-01-01,3,3,2,3


In [12]:
df

,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
Date,,,,,,,,,,,,,,,
2022-01-01,S001,P0001,1,1,195,102,252,72.72,5,2,0,85.73,3,0,115
2022-01-01,S004,P0013,3,3,136,104,385,20.24,10,2,0,18.90,3,0,110
2022-01-01,S004,P0012,1,3,111,111,113,118.15,0,2,0,133.46,3,0,103
2022-01-01,S004,P0011,0,3,195,60,293,52.89,0,2,0,62.29,3,0,61
2022-01-01,S004,P0010,3,3,223,120,597,30.02,0,2,0,29.15,3,0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-30,S002,P0008,3,2,85,81,94,26.98,0,2,0,24.65,3,0,96
2024-01-30,S002,P0007,4,2,414,126,0,26.84,10,2,0,29.55,3,0,109
2024-01-30,S002,P0006,0,2,98,96,117,63.58,10,2,0,76.22,3,0,118


In [13]:
df['Price_lag_1'] = df['Price'].shift(1)
df['Price_lag_2'] = df['Price'].shift(2)
df['Price_lag_3'] = df['Price'].shift(3)

In [14]:
# Drop rows with missing lag values
df.dropna(subset=['Price_lag_1', 'Price_lag_2', 'Price_lag_3'], inplace=True)

# Reset index if you want
df.reset_index(inplace=True)

In [15]:
# Choose useful features for forecasting Demand
selected_cols = [
    'Date', 'Region', 'Category', 'Inventory Level', 'Units Ordered',
    'Discount', 'Promotion', 'Seasonality', 'Epidemic',
    'Price_lag_1', 'Price_lag_2', 'Price_lag_3', 'Demand'
]
df = df[selected_cols]

df.head()

,Date,Region,Category,Inventory Level,Units Ordered,Discount,Promotion,Seasonality,Epidemic,Price_lag_1,Price_lag_2,Price_lag_3,Demand
0,2022-01-01,3,0,195,293,0,0,3,0,118.15,20.24,72.72,61
1,2022-01-01,3,3,223,597,0,0,3,0,52.89,118.15,20.24,128
2,2022-01-01,3,0,206,299,5,0,3,0,30.02,52.89,118.15,98
3,2022-01-01,3,2,281,0,0,0,3,0,110.47,30.02,52.89,80
4,2022-01-01,3,3,185,253,5,0,3,0,100.64,110.47,30.02,65


In [16]:
# Sort by Date (ensure time order)
df = df.sort_values('Date')

# Split 80% train, 20% test
split = int(0.8 * len(df))
train = df.iloc[:split]
test = df.iloc[split:]

print("Train set size:", len(train))
print("Test set size:", len(test))

Train set size: 60797
Test set size: 15200


##Baseline

In [23]:
exog_features = ['Price_lag_1', 'Price_lag_2', 'Price_lag_3',
                 'Discount', 'Promotion', 'Seasonality', 'Epidemic', 'Region', 'Category', 'Inventory Level', 'Units Ordered']

X_train = train[exog_features]
X_test = test[exog_features]

In [20]:
y_train = train['Demand']
y_test = test['Demand']

In [24]:
# Train baseline ARIMAX model
model = ARIMA(endog=y_train, exog=X_train, order=(1, 1, 1))

In [25]:
model_fit = model.fit()
pred_baseline = model_fit.forecast(steps=len(test), exog=X_test)

In [26]:
# Calculate error metrics
mae = mean_absolute_error(y_test, pred_baseline)
rmse = np.sqrt(mean_squared_error(y_test, pred_baseline))

# MAPE
mape = np.mean(np.abs((y_test - pred_baseline) / y_test)) * 100

# R2 Score
r2 = r2_score(y_test, pred_baseline)

# Print results
print("📈 Baseline ARIMAX(1,1,1) Performance Metrics:")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.3f}")

📈 Baseline ARIMAX(1,1,1) Performance Metrics:
MAPE : 60.32%
R²   : 0.281


## Fine tune


In [27]:
import itertools
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [28]:
# Define the range for (p, d, q)
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_score = float("inf")
best_order = None
best_metrics = {}

In [ ]:
# Grid Search
for order in pdq:
    try:
        model = ARIMA(endog=y_train, exog=X_train, order=order)
        model_fit = model.fit()
        pred = model_fit.forecast(steps=len(test), exog=X_test)

        # Calculate metrics
        mae = mean_absolute_error(y_test, pred)
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        mape = np.mean(np.abs((y_test - pred) / y_test)) * 100

        # Use RMSE as main selection metric
        if rmse < best_score:
            best_score = rmse
            best_order = order
            best_metrics = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}
    except Exception as e:
        continue

In [ ]:
print(f"Best MAPE: {best_metrics['MAPE']:.2f}%")

In [ ]:
# Refit with the best parameters
best_model = ARIMA(endog=y_train, exog=X_train, order=best_order)
best_fit = best_model.fit()

# Forecast using the tuned model
pred_finetuned = best_fit.forecast(steps=len(test), exog=X_test)

# Compute evaluation metrics
mae_fine = mean_absolute_error(y_test, pred_finetuned)
rmse_fine = np.sqrt(mean_squared_error(y_test, pred_finetuned))
mape_fine = np.mean(np.abs((y_test - pred_finetuned) / y_test)) * 100
r2_fine = r2_score(y_test, pred_finetuned)

In [ ]:
print("Fine-Tuned ARIMAX Model Performance:")
print(f"MAPE : {mape_fine:.2f}%")
print(f"R2  : {r2_fine:.3f}")